In [43]:
import re
from jamo import h2j, j2hcj, j2h
import pickle
from gensim.models import Word2Vec
import numpy as np
# 불러올때
with open('data/result.txt', 'rb') as f:
    result = pickle.load(f)

# ---(박사님 같이) 코사인유사도 구하기 연습------

In [ ]:
test_m = np.array([1,2, 3, 4, 5])
test_m

In [ ]:
test_m = np.diag(test_m)
test_m

In [ ]:
Q = np.array([1,0,0,0, 0])
Q

In [ ]:
Q.reshape(5, -1)

In [ ]:
dot_pro.shape

In [ ]:
dot_pro

In [ ]:
Q.shape

In [ ]:
Q

In [ ]:
dot_pro = test_m.dot(Q)

In [ ]:
np.linalg.norm(test_m, axis=1)#길이

In [ ]:
np.linalg.norm(Q) #길이

In [ ]:
np.linalg.norm(test_m, axis=1) * np.linalg.norm(Q) #길

In [ ]:
sim = dot_pro / np.linalg.norm(test_m, axis=1) * np.linalg.norm(Q) #길

In [ ]:
sim.argmax()

# --------------

### -----문장간 코사인유사도 찾기(좋은 임베딩 파라미터 찾기------

In [3]:
result[:10]

[' 반갑습니다',
 ' 반갑습니다',
 ' 이비에스 친구 여러분 여러분 함께 중학 뉴런 수학 삼하를 강의 된손석민 입니다',
 ' 이비에스  여러분 여러분 함께 중학 뉴런 수학 삼할을 강의 하기됨손성민 입니다',
 ' 학년 뉴런 통해 서도 만났었는데요 여러분 이렇게 다시 만나게 되어서 정말 반갑습니다',
 ' 학년 이런을 통해 서도 만났었는데 여러분 이렇게 다시 만나게 되어서 정말 반갑습니다',
 ' 앞 삼하 과정 어떻게 진행 되는지에 대해 서는 여러분 오리엔테이션 강의 통해 확인 할수있을거고요 오리엔테이션 강의 반드시 듣고오셔야 여러분 이삼 학년 과정 이하과정특히 어떻게 수업 되는지 확인 할수있을 겁니다',
 ' 앞 삼 과정 어떻게 진행 되는지에 대해 서는 여러분 오리엔테이션 강의 통해 확인 할수있을거구요 오리엔테이션 강의 반드시 듣고오셔야 여러분 이삼 학년 과정 하마종이접기 어떻게 수업 되는지 확인 할수있을 겁니다',
 ' 일 단삼 학년 과정 같은 경우 크게 삼 개 대단원 이루어져있는데요 첫 삼각비 단원두번째 가원 성질 세번 통계 이렇게 이루어져있습니다',
 ' 일 단삼 학년 가정 같은 경우 크게 세 개 대단원 이루어져있는데요 첫 삼각비 단어거두 가원 성질 세번 통계 이렇게 이루어져있습니다']

In [4]:
#ebs, google 문장들 쪼개서
result_gensim_input = [_.split() for _ in result if _ != '']
print(len(result_gensim_input))
print(result_gensim_input[:3])

4162
[['반갑습니다'], ['반갑습니다'], ['이비에스', '친구', '여러분', '여러분', '함께', '중학', '뉴런', '수학', '삼하를', '강의', '된손석민', '입니다']]


In [5]:
#ebs 문장들 안쪼개서
result_tokenizer_input = [v for i, v in enumerate(result) if i%2 == 0 and v != '']
print(len(result_tokenizer_input))
print(result_tokenizer_input[:3])

2081
[' 반갑습니다', ' 이비에스 친구 여러분 여러분 함께 중학 뉴런 수학 삼하를 강의 된손석민 입니다', ' 학년 뉴런 통해 서도 만났었는데요 여러분 이렇게 다시 만나게 되어서 정말 반갑습니다']


In [6]:
#ebs 문장들 쪼개서
ebs_in_result_for_getting_max_len = [v for i, v in enumerate(result_gensim_input) if i%2 == 0 and v != []]
print(len(ebs_in_result_for_getting_max_len))
print(ebs_in_result_for_getting_max_len[:3])

2081
[['반갑습니다'], ['이비에스', '친구', '여러분', '여러분', '함께', '중학', '뉴런', '수학', '삼하를', '강의', '된손석민', '입니다'], ['학년', '뉴런', '통해', '서도', '만났었는데요', '여러분', '이렇게', '다시', '만나게', '되어서', '정말', '반갑습니다']]


In [27]:
ebs_in_result_gensim_input = [v for i, v in enumerate(result_gensim_input) if i%2 == 0 and v != [] and result_gensim_input[i+1] != []]
print(len(ebs_in_result_gensim_input))
print(ebs_in_result_gensim_input[:3])

2079
[['반갑습니다'], ['이비에스', '친구', '여러분', '여러분', '함께', '중학', '뉴런', '수학', '삼하를', '강의', '된손석민', '입니다'], ['학년', '뉴런', '통해', '서도', '만났었는데요', '여러분', '이렇게', '다시', '만나게', '되어서', '정말', '반갑습니다']]


In [28]:
google_in_result_gensim_input = [v for i, v in enumerate(result_gensim_input) if i%2 == 1 and v != []]
print(len(google_in_result_gensim_input))
print(google_in_result_gensim_input[:3])

2079
[['반갑습니다'], ['이비에스', '여러분', '여러분', '함께', '중학', '뉴런', '수학', '삼할을', '강의', '하기됨손성민', '입니다'], ['학년', '이런을', '통해', '서도', '만났었는데', '여러분', '이렇게', '다시', '만나게', '되어서', '정말', '반갑습니다']]


In [29]:
embedding_size = 30
window_size = 1

model = Word2Vec(
                result_gensim_input, 
                window = window_size,
                size =embedding_size,
                min_count=1,
                workers = 10
                )

def get_s_vec(s_list, embedding_size):
    s_vec = np.zeros(embedding_size)
    for t in s_list:
        s_vec += model[t]
    s_vec /= len(s_list)
    return s_vec

In [30]:
# Generate EBS string vectors matrix
ebs_str_arr = np.zeros((len(ebs_in_result_gensim_input), embedding_size))
for i, s_list in enumerate(ebs_in_result_gensim_input):
    ebs_str_arr[i] = get_s_vec(s_list, embedding_size)
ebs_vec_norms = np.linalg.norm(ebs_str_arr, axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [ ]:
#수고하셨습니다가 들어가는 문장 인덱스
lec_last_str_idx = '104 192 299 407 517 617 715 780 896 1004 1091 1214 1340 1531 1588 1626 1657 1700 1742 1797 1845 1877 1931 1993 2035 2078'
lec_last_str_idx = [int(_) for _ in lec_last_str_idx.split()]

In [116]:
#Query comparison
cos_sims_bool = np.zeros(len(google_in_result_gensim_input))

i = 1
ebs = ''
google = ''
for g_idx, Q_str in enumerate(google_in_result_gensim_input):
    Q_vec = get_s_vec(Q_str, embedding_size)
#     Q_vec = Q_vec.reshape(embedding_size, -1)
    dot_products = ebs_str_arr.dot(Q_vec)

    Q_vec_norm = np.linalg.norm(Q_vec)
    norm_productions = ebs_vec_norms * Q_vec_norm
    cos_sims = dot_products / norm_productions
    
#     print(g_idx, '   ', cos_sims.argmax())
#     print(ebs_in_result_gensim_input[cos_sims.argmax()])
    
    if g_idx == cos_sims.argmax():
        cos_sims_bool[g_idx] = 1
    
    ebs += '\n' + ' '.join(ebs_in_result_gensim_input[cos_sims.argmax()])
    google += '\n' + ' '.join(Q_str)
    if g_idx in lec_last_str_idx:
        ebs_save_name = 'result_conversion_' + '{0:05d}'.format(i) + '.txt'
        google_save_name = 'result_google_' + '{0:05d}'.format(i) + '.txt'
        with open('data/'+ebs_save_name, 'w') as text:
            text.write(ebs)
        with open('data/'+google_save_name, 'w') as text:
            text.write(google)
        ebs = ''
        google = ''
        i+=1
    
diff_num = np.unique(cos_sims_bool, return_counts=True)[1][0]
same_num = np.unique(cos_sims_bool, return_counts=True)[1][1]
print(same_num/(diff_num+same_num) * 100)

    

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


82.68398268398268
